# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [3]:
y.mean()

0.5

In [4]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [5]:
In_train = pd.DataFrame(columns=X.columns,data=X_train)
In_test = pd.DataFrame(columns=X.columns,data=X_test)
Out_train = pd.DataFrame(columns=['Popular'],data=y_train)
Out_test = pd.DataFrame(columns=['Popular'],data=y_test)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

models = {'dt': DecisionTreeClassifier(),
          'lr': LogisticRegression(),
          }

In [8]:
# Train all the models
for model in models.keys():
    models[model].fit(X_train, y_train)

In [9]:
# predict test for each model
y_pred = pd.DataFrame(index=y_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)    

In [10]:
y_pred.head()

,dt,lr
1483,0,1
2185,0,1
2520,1,1
3721,0,1
3727,0,0


In [11]:
# Evaluate each model

#Accuracy:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

for model in models.keys():
    print(model,'\tAccuracy:',accuracy_score(y_pred[model],y_test),'\tF1_Score',f1_score(y_pred[model],y_test))

dt 	Accuracy: 0.5533333333333333 	F1_Score 0.5557029177718832
lr 	Accuracy: 0.622 	F1_Score 0.6043265875785067


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [12]:
import numpy as np

In [13]:
n_estimators = 100

# set a seed for reproducibility
np.random.seed(123)

n_samples = 300

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(300)]

In [14]:
# 100 Decision Trees where max_depth=None
from sklearn.tree import DecisionTreeClassifier

np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

trees = {}
for i in range(n_estimators):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [15]:
# 100 Decision Trees where max_depth=2
np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

trees2 = {}
for i in range(100,200):
    trees2[i-100] = DecisionTreeClassifier(max_features="sqrt", max_depth=2, random_state=seeds[i-100])
    trees2[i-100].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])


In [16]:
# 100 Logistic Regressions
from sklearn.linear_model import LogisticRegression

np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

regs = {}
for i in range(200,300):
    regs[i-200] = LogisticRegression(random_state=seeds[i-200])
    regs[i-200].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [17]:
# Predict 
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(300)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = trees[i].predict(X_test)

In [18]:
for i in range(100,200):
    y_pred_df.iloc[:, i] = trees2[i-100].predict(X_test)

In [19]:
for i in range(200,300):
    y_pred_df.iloc[:, i] = regs[i-200].predict(X_test)
y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,0,0,0,1,1,0,1,1,0,1,...,0,1,1,1,1,1,1,1,1,1
2185,0,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2520,1,1,1,1,0,1,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
3721,0,1,1,0,0,1,0,1,1,1,...,1,0,1,0,1,1,1,1,1,1
3727,1,0,0,1,0,1,1,0,1,1,...,0,1,0,0,0,0,0,0,0,0


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [20]:
# Calculate the aprox probability:
y_pred2 = (y_pred_df.sum(axis=1) >= (n_samples / 2)).astype(np.int)

In [21]:
from sklearn import metrics

In [22]:
print('F1 Score:\t',metrics.f1_score(y_pred2, y_test),'\nAccuracy:\t',metrics.accuracy_score(y_pred2, y_test))

F1 Score:	 0.6260273972602739 
Accuracy:	 0.636


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score.

In [23]:
# Calculate the aprox probability:
cols=['Threshold','F1 Score','Accuracy']
MVoting=pd.DataFrame(columns=cols,data=[])
k=MVoting.shape[0]

thresh = range(50,90,2)
for th in thresh:
    y_pred2 = (y_pred_df.sum(axis=1) >= (n_samples * th / 100)).astype(np.int)
    #print(th/100,'\tF1 Score:\t',metrics.f1_score(y_pred2, y_test),'Accuracy:\t',metrics.accuracy_score(y_pred2, y_test))
    MVoting.loc[k] = [th/100,metrics.f1_score(y_pred2, y_test),metrics.accuracy_score(y_pred2, y_test)]
    k+=1
MVoting

,Threshold,F1 Score,Accuracy
0,0.50,0.626027,0.636000
1,0.52,0.616028,0.632667
2,0.54,0.605114,0.629333
3,0.56,0.597534,0.630000
4,0.58,0.582840,0.624000
5,0.60,0.577677,0.626667
6,0.62,0.561295,0.620667
7,0.64,0.549606,0.618667
8,0.66,0.532794,0.615333
9,0.68,0.516236,0.612667


The best Score:

In [24]:
MVoting[MVoting['F1 Score']==np.max(MVoting['F1 Score'])]

,Threshold,F1 Score,Accuracy
0,0.5,0.626027,0.636


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

**Select each oob sample**

In [25]:
samples_oob = []
# show the "out-of-bag" observations for each sample
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

In [26]:
errors = np.zeros(300)

for i in range(n_estimators):
    y_pred_ = trees[i].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)

In [27]:
for i in range(100,200):
    y_pred_ = trees2[i-100].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)

In [28]:
for i in range(200,300):
    y_pred_ = trees2[i-200].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)

In [29]:
alpha = (1 - errors) / (1 - errors).sum()

In [30]:
weighted_sum_1 = ((y_pred_df) * alpha).sum(axis=1)

In [31]:
y_pred = (weighted_sum_1 >= 0.5).astype(np.int)

print('F1 Score:\t',metrics.f1_score(y_pred, y_test),'\nAccuracy:\t',metrics.accuracy_score(y_pred, y_test))

F1 Score:	 0.6241426611796983 
Accuracy:	 0.6346666666666667


# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [32]:
# Calculate the aprox probability:
cols=['Threshold','F1 Score','Accuracy']
WVoting=pd.DataFrame(columns=cols,data=[])
k=WVoting.shape[0]

thresh = range(50,90,2)
for th in thresh:
    y_pred = (weighted_sum_1 >= (th/100)).astype(np.int)
    WVoting.loc[k] = [th/100,metrics.f1_score(y_pred, y_test),metrics.accuracy_score(y_pred, y_test)]
    k+=1
WVoting

,Threshold,F1 Score,Accuracy
0,0.50,0.624143,0.634667
1,0.52,0.616562,0.632667
2,0.54,0.605674,0.629333
3,0.56,0.599278,0.630000
4,0.58,0.587454,0.627333
5,0.60,0.577444,0.625333
6,0.62,0.561538,0.620000
7,0.64,0.548589,0.616000
8,0.66,0.539759,0.618000
9,0.68,0.519008,0.612000


The best F1 Score:

In [33]:
WVoting[WVoting['F1 Score']==np.max(WVoting['F1 Score'])]

,Threshold,F1 Score,Accuracy
0,0.5,0.624143,0.634667


# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers (stacking)

Modify the probability threshold such that maximizes the F1-Score

In [34]:
y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,0,0,0,1,1,0,1,1,0,1,...,0,1,1,1,1,1,1,1,1,1
2185,0,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2520,1,1,1,1,0,1,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
3721,0,1,1,0,0,1,0,1,1,1,...,1,0,1,0,1,1,1,1,1,1
3727,1,0,0,1,0,1,1,0,1,1,...,0,1,0,0,0,0,0,0,0,0


In [35]:
# Define the input as the all outputs from de previous models
X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(300)))
for i in range(n_estimators):
    X_train_2.iloc[:, i] = trees[i].predict(X_train)

In [36]:
for i in range(100,200):
    X_train_2.iloc[:, i] = trees2[i-100].predict(X_train)

In [37]:
for i in range(200,300):
    X_train_2.iloc[:, i] = regs[i-200].predict(X_train)
X_train_2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2420,1,1,1,0,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
1225,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,0,1,1
324,0,1,0,0,0,0,0,1,0,0,...,1,1,1,1,1,1,0,1,1,1
5863,1,1,1,1,1,1,1,0,1,1,...,1,1,1,0,1,1,0,1,1,1
86,1,0,1,1,0,1,1,1,1,0,...,1,0,0,1,0,1,1,1,1,0


In [38]:
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv = 5 )
lr.fit(X_train_2, y_train)
y_pred = lr.predict(y_pred_df)
print('F1 Score:\t',metrics.f1_score(y_pred, y_test),'\nAccuracy:\t',metrics.accuracy_score(y_pred, y_test))

F1 Score:	 0.6644693473961767 
Accuracy:	 0.6606666666666666


In [39]:
# Calculate the probability:
cols=['Threshold','F1 Score']
F1=pd.DataFrame(columns=cols,data=[])
k=F1.shape[0]

for th in thresh:
    lr.fit(X_train_2, y_train)
    y_pred = np.where(lr.predict_proba(y_pred_df)[:,1] > th/100, 1, 0)
    F1.loc[k] = [th/100,metrics.f1_score(y_pred, y_test)]
    k+=1

In [40]:
F1

,Threshold,F1 Score
0,0.50,0.664469
1,0.52,0.642955
2,0.54,0.628815
3,0.56,0.619883
4,0.58,0.595004
5,0.60,0.577255
6,0.62,0.533003
7,0.64,0.496552
8,0.66,0.462780
9,0.68,0.414313


The best F1 Score:

In [41]:
F1[F1['F1 Score']==np.max(F1['F1 Score'])]

,Threshold,F1 Score
0,0.5,0.664469
